In [0]:
!pip uninstall --quiet --yes tensorflow
!pip install --quiet tensorflow-gpu==1.13.1
!pip install --quiet tensorflow-hub
!pip install --quiet seaborn
!pip install --quiet tf-sentencepiece

In [0]:
from keras.models import Sequential
from keras.layers import Dense, Activation, BatchNormalization
import keras
import tensorflow as tf


In [0]:
# Generate data
import numpy as np
import io

def load_embeddings(emb_path):
  data = []
  with io.open(emb_path, 'r', encoding='utf-8', newline='\n', errors='ignore') as f:
        for i, line in enumerate(f):
                word, vect = line.rstrip().split(' ', 1)

                word = word.lower()
                vect = np.fromstring(vect, sep=' ')
                if np.linalg.norm(vect) == 0:  # avoid to have null embeddings
                    vect[0] = 0.01

                else:
                    if not vect.shape == (300,):
                        print("Invalid dimension (%i) for word '%s' in line %i."
                                       % (vect.shape[0], word, i))
                        continue
                    assert vect.shape == (300,), i
                    data.append(vect)
  return data

d = np.array(load_embeddings('/content/supervision-init-full-new-dataset-en-emb.vec'))
l = np.array(load_embeddings('/content/supervision-init-full-new-dataset-es-emb.vec'))

data = d[:,]
labels = l[:,]
data_test = np.array(load_embeddings('/content/test-dict-only-words-en-emb.vec'))
data_labels = np.array(load_embeddings('/content/test-dict-only-words-es-emb.vec'))

src_embeddings = np.array(np.load('/content/200k-emb-en.npy'))
tgt_embeddings = np.array(np.load('/content/200k-emb-es.npy'))
print(data.shape)
print(labels.shape)
print(src_embeddings.shape)
print(tgt_embeddings.shape)

In [0]:
def cosine_similarity(y_true, y_pred):
  a = y_true 
  b = y_pred
  normalize_a = tf.nn.l2_normalize(a)        
  normalize_b = tf.nn.l2_normalize(b)
  cos_similarity=tf.reduce_sum(tf.multiply(normalize_b, normalize_a))
  return cos_similarity

In [0]:
from tensorflow.python.keras import metrics


model = Sequential()

FC_layer = Dense(300, activation='tanh', input_dim=300, use_bias=True, kernel_initializer = keras.initializers.glorot_uniform(seed=None))
# FC_layer = Dense(300, activation='sigmoid', input_dim=300, use_bias=True, kernel_initializer = keras.initializers.glorot_uniform(seed=None))
# FC_layer = Dense(300, activation='relu', input_dim=300, use_bias=True, kernel_initializer = keras.initializers.glorot_uniform(seed=None))
# FC_layer = Dense(300, activation=keras.layers.LeakyReLU(alpha=0.3), input_dim=300, use_bias=True, kernel_initializer = keras.initializers.glorot_uniform(seed=None))
# BN_layer = BatchNormalization()

model.add(FC_layer)



adam = keras.optimizers.Adam(lr=1e-2, beta_1=0.5, beta_2=0.999)
model.compile(optimizer=adam,
              loss='mse',
              metrics=[cosine_similarity, 'acc'])



model.fit(data[:, :], labels[:,:], epochs=80, batch_size=128)
model.evaluate(data_test, data_labels)
TranslatedX = model.predict(src_embeddings)
np.save('TranslatedX-tanh', TranslatedX)

In [0]:
#linear (supervised)

from tensorflow.python.keras import metrics

model = Sequential()
FC_layer = Dense(300, activation=None, input_dim=300, use_bias=False, kernel_initializer = keras.initializers.glorot_uniform(seed=None))


model.add(FC_layer)

adam = keras.optimizers.Adam(lr=1e-2, beta_1=0.5, beta_2=0.999)
model.compile(optimizer=adam,
              loss='mse',
              metrics=[cosine_similarity, 'acc'])

model.fit(data, labels, epochs=80, batch_size=128)
model.evaluate(data_test, data_labels)

In [0]:
#init with linear
T = np.load('/content/en_es_T.npy')
T = np.transpose(T)


def my_init(shape, dtype=None):
    return T
  

model = Sequential()
FC_layer = Dense(300, activation='relu', input_dim=300, use_bias=False, kernel_initializer=my_init)
model.add(FC_layer)

adam = keras.optimizers.Adam(lr=1e-2, beta_1=0.5, beta_2=0.999)
model.compile(optimizer=adam,
              loss='mse',
              metrics=[cosine_similarity, 'acc'])
model.fit(data, labels, epochs=50, batch_size=128)

model.evaluate(data_test, data_labels)

In [0]:
#deeper
from tensorflow.python.keras import metrics


model = Sequential()

FC_layer = Dense(300, activation='tanh', input_dim=300, use_bias=True, kernel_initializer = keras.initializers.glorot_uniform(seed=None))
BN_layer = BatchNormalization()

model.add(FC_layer)
model.add(BN_layer)
model.add(FC_layer)
model.add(BN_layer)
model.add(FC_layer)
model.add(BN_layer)


adam = keras.optimizers.Adam(lr=1e-2, beta_1=0.5, beta_2=0.999)
model.compile(optimizer=adam,
              loss='mse',
              metrics=[cosine_similarity, 'acc'])

model.fit(data, labels, epochs=50, batch_size=128)
model.evaluate(data_test, data_labels)

In [0]:
#down-up
from tensorflow.python.keras import metrics


model = Sequential()


model.add(Dense(50, activation='relu', input_dim=300, use_bias=True, kernel_initializer = keras.initializers.glorot_uniform(seed=None)))
model.add( BatchNormalization())
model.add(Dense(300, activation='relu', input_dim=50, use_bias=True, kernel_initializer = keras.initializers.glorot_uniform(seed=None)))
model.add( BatchNormalization())
model.add(Dense(300, activation='relu', input_dim=300, use_bias=True, kernel_initializer = keras.initializers.glorot_uniform(seed=None)))
model.add( BatchNormalization())




adam = keras.optimizers.Adam(lr=1e-2, beta_1=0.5, beta_2=0.999)

model.compile(optimizer=adam,
              loss='mse',
              metrics=[cosine_similarity, 'acc'])

model.fit(data[:,], labels[:,], epochs=200, batch_size=128)
model.evaluate(data_test, data_labels)

In [0]:
#linear+relu
from tensorflow.python.keras import metrics


model = Sequential()

FC_layer = Dense(300, activation='linear', input_dim=300, use_bias=True, kernel_initializer = keras.initializers.glorot_uniform(seed=None))
BN_layer = BatchNormalization()
FC_layer_relu = Dense(300, activation='relu', input_dim=300, use_bias=True, kernel_initializer = keras.initializers.glorot_uniform(seed=None))

model.add(FC_layer)
model.add(BN_layer)
model.add(FC_layer_relu)
model.add(BN_layer)




#just changing the lr
adam = keras.optimizers.Adam(lr=1e-2, beta_1=0.5, beta_2=0.999)
model.compile(optimizer=adam,
              loss='mse',
              metrics=[cosine_similarity, 'acc'])

model.fit(data, labels, epochs=200, batch_size=128)
model.evaluate(data_test, data_labels)

In [0]:
#sgd
from tensorflow.python.keras import metrics


model = Sequential()

FC_layer = Dense(300, activation='linear', input_dim=300, use_bias=True, kernel_initializer = keras.initializers.glorot_uniform(seed=None))
BN_layer = BatchNormalization()
FC_layer_relu = Dense(300, activation='relu', input_dim=300, use_bias=True, kernel_initializer = keras.initializers.glorot_uniform(seed=None))

model.add(FC_layer)
model.add(BN_layer)
model.add(FC_layer_relu)




sgd = keras.optimizers.SGD(lr=1e-2, decay=1e-6, momentum=0.9, nesterov=True)

model.compile(optimizer=sgd,
              loss='mse',
              metrics=[cosine_similarity, 'acc'])

model.fit(data, labels, epochs=200, batch_size=128)
model.evaluate(data_test, data_labels)

In [0]:
#just changing the lr
from tensorflow.python.keras import metrics


model = Sequential()

FC_layer = Dense(300, activation=None, input_dim=300, use_bias=True, kernel_initializer = keras.initializers.glorot_uniform(seed=None))

model.add(FC_layer)


#just changing the lr
adam = keras.optimizers.Adam(lr=1e-4, beta_1=0.5, beta_2=0.999)
model.compile(optimizer=adam,
              loss='mse',
              metrics=[cosine_similarity, 'acc'])

model.fit(data, labels, epochs=50, batch_size=128)